In [90]:
import os
os.chdir('C:/Users/islam/Downloads/kidney-Disease-classification-Deep-learning-project/research')

In [91]:
%pwd

'C:\\Users\\islam\\Downloads\\kidney-Disease-classification-Deep-learning-project\\research'

In [92]:
os.chdir("../")

In [93]:
%pwd

'C:\\Users\\islam\\Downloads\\kidney-Disease-classification-Deep-learning-project'

In [94]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/tarek-0566/kidney-Disease-classification-Deep-learning-project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="tarek-0566"
os.environ["MLFLOW_TRACKING_PASSWORD"]="da3b0354d103ab90d8f0b79c7f1d922ff035175d"

In [105]:
pip install tensorflow==2.12.0

Note: you may need to restart the kernel to use updated packages.


In [104]:
import tensorflow as tf

In [96]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [97]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [98]:
import sys
import os
import joblib

# Add the src folder to the Python path
project_dir = r'C:/Users/islam/Downloads/kidney-Disease-classification-Deep-learning-project/src'
sys.path.append(project_dir)

from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [99]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="C:/Users/islam/Downloads/kidney-Disease-classification-Deep-learning-project/artifacts/training/model.h5",
            training_data="C:/Users/islam/Downloads/kidney-Disease-classification-Deep-learning-project/artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/tarek-0566/kidney-Disease-classification-Deep-learning-project.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [25]:
pip install mlflow

  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/26.7 MB ? eta -:--:--
   --- ------------------------------------ 2.4/26.7 MB 12.2 MB/s eta 0:00:02
   ------- -------------------------------- 5.0/26.7 MB 12.1 MB/s eta 0:00:02
   ---------- ----------------------------- 7.3/26.7 MB 11.9 MB/s eta 0:00:02
   -------------- ------------------------- 10.0/26.7 MB 11.7 MB/s eta 0:00:02
   ------------------ --------------------- 12.6/26.7 MB 11.8 MB/s eta 0:00:02
   ---------------------- ----------------- 14.9/26.7 MB 11.7 MB/s eta 0:00:02
   -------------------------- ------------- 17.6/26.7 MB 11.9 MB/s eta 0:00:01
   ----------------------------- ---------- 19.9/26.7 MB 11.8 MB/s eta 0:00:01
   --------------------------------- ------ 22.5/26.7 MB 11.8 MB/s eta 0:00:01
   ------------------------------------- -- 24.9/26.7 MB 11.8 MB/s eta 0:00:01
   ---------------------------------------  26.7/26.7 MB 11.8 MB/s eta 0:0

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.13.0 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.12.2 which is incompatible.


In [100]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [101]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [102]:
import os
from pathlib import Path

CONFIG_FILE_PATH = Path(os.path.abspath("config/config.yaml"))
PARAMS_FILE_PATH = Path(os.path.abspath("params.yaml"))

In [103]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-10-29 14:12:59,712: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-10-29 14:12:59,723: INFO: common: yaml file: params.yaml loaded successfully]
[2024-10-29 14:12:59,725: INFO: common: created directory at: artifacts]
Found 139 images belonging to 2 classes.
9/9 [==============================] - 27s 3s/step - loss: 2.1054 - accuracy: 0.5036
[2024-10-29 14:13:27,912: INFO: common: json file saved at: scores.json]


2024/10/29 14:13:28 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-10-29 14:13:31,406: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\islam\AppData\Local\Temp\tmpurzwlaq5\model\data\model\assets
[2024-10-29 14:13:33,259: INFO: builder_impl: Assets written to: C:\Users\islam\AppData\Local\Temp\tmpurzwlaq5\model\data\model\assets]


2024/10/29 14:13:49 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2024/10/29 14:14:16 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 3
Created version '3' of model 'VGG16Model'.
2024/10/29 14:14:16 INFO mlflow.tracking._tracking_service.client: 🏃 View run intrigued-sheep-465 at: https://dagshub.com/tarek-0566/kidney-Disease-classification-Deep-learning-project.mlflow/#/experiments/0/runs/9740da798d93457599e2248ecda6bc58.
2024/10/29 14:14:16 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/tarek-0566/kidney-Disease-classification-Deep-learning-project.mlflow/#/experiments/0.
